In [1]:
import pandas as pd
import pyspark.sql.dataframe as sp
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff

spark = SparkSession.builder.appName('LeetCode').getOrCreate()

24/12/27 15:56:03 WARN Utils: Your hostname, madiv resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
24/12/27 15:56:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/27 15:56:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/27 15:56:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
spark_Weather = spark.read.option('header', True).option('delimiter', ',').csv('Weather.csv')
pandas_Weather = pd.read_csv('Weather.csv')

spark_Weather.show()
pandas_Weather.head()

+---+----------+-----------+
| id|recordDate|temperature|
+---+----------+-----------+
|  1|2015-01-01|         10|
|  2|2015-01-02|         25|
|  3|2015-01-03|         20|
|  4|2015-01-04|         30|
+---+----------+-----------+



,id,recordDate,temperature
0,1,2015-01-01,10
1,2,2015-01-02,25
2,3,2015-01-03,20
3,4,2015-01-04,30


### SQL

In [3]:
def rising_temperature(weather: sp.DataFrame) -> sp.DataFrame:
    weather.createOrReplaceTempView('Weather')
    sqlQuery = \
    '''
    SELECT w1.id FROM Weather w1 JOIN Weather w2
    ON w1.temperature > w2.temperature AND 
    DATEDIFF(w1.recordDate, w2.recordDate) = 1
    '''
    
    ids_With_Rising_Temperature = spark.sql(sqlQuery = sqlQuery)
    return ids_With_Rising_Temperature

output = rising_temperature(weather = spark_Weather)
output.show()

+---+
| id|
+---+
|  2|
|  4|
+---+



### PySpark

In [4]:
def rising_temperature(weather: sp.DataFrame) -> sp.DataFrame:
    ids_With_Rising_Temp = weather.alias('w1').join(weather.alias('w2'), 
                                        on = ((col('w1.temperature') > col('w2.temperature')) &
                                              (datediff(col('w1.recordDate'), col('w2.recordDate')) == 1)), 
                                        how = 'inner')\
                                    .select(col('w1.id'))
    return ids_With_Rising_Temp

output = rising_temperature(weather = spark_Weather)
output.show()

+---+
| id|
+---+
|  2|
|  4|
+---+



### Pandas

In [5]:
def rising_temperature(weather: pd.DataFrame) -> pd.DataFrame:
    w1 = weather.copy()
    w2 = weather.copy()
    
    data_With_Increasing_Temp = w1.merge(w2, on = ((w1.temperature > w2.temperature)), 
                                         how = 'inner')
    
    data_With_Increasing_Temp['DateDiff'] = (pd.to_datetime(data_With_Increasing_Temp.recordDate_x) - 
                                                pd.to_datetime(data_With_Increasing_Temp.recordDate_y))
    
    data_With_Increasing_Temp['TempDiff'] = (data_With_Increasing_Temp.temperature_x - 
                                             data_With_Increasing_Temp.temperature_y)
    
    ids_With_Increasing_Temp = data_With_Increasing_Temp[(data_With_Increasing_Temp.DateDiff == '1 Days') & 
                                                         (data_With_Increasing_Temp.TempDiff > 0)]
    
    ids_With_Increasing_Temp.rename(columns = {'id_x':'id'}, inplace = True)
    
    return ids_With_Increasing_Temp[['id']]

output = rising_temperature(weather = pandas_Weather)
output.head(100)

/tmp/ipykernel_23942/470766803.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ids_With_Increasing_Temp.rename(columns = {'id_x':'id'}, inplace = True)


,id
4,2
14,4


In [6]:
def rising_temperature(weather: pd.DataFrame) -> pd.DataFrame:
    # Sort the weather DataFrame by recordDate to ensure proper order
    weather = weather.sort_values(by='recordDate')
    
    # Calculate the difference between consecutive recordDates
    weather['date_diff'] = pd.to_datetime(weather['recordDate']).diff().dt.days
    
    # Use shift() to get the previous day's temperature
    weather['prev_temp'] = weather['temperature'].shift(1)
    
    # Filter for rows where the temperature is greater than the previous day
    # and the date difference is exactly 1 day
    result = weather[(weather['temperature'] > weather['prev_temp']) & 
                     (weather['date_diff'] == 1)]
    
    # Return the 'id' of the rows that meet the condition
    return result[['id']]

# Call the function
output = rising_temperature(pandas_Weather)

output.head(100)

,id
1,2
3,4
